# Neo4j Generative AI - Data Loading
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/neo4j-product-examples/genai-workshop/blob/main/data-load.ipynb)

This workshop will teach you to how to use Neo4j for Graph-Powered Retrieval-Augmented Generation (GraphRAG) to enhance GenAI and improve response quality for real-world applications.

GenAI, despite its potential, faces challenges like hallucination and lack of domain knowledge. GraphRAG addresses these issues by combining vector search with knowledge graphs and data science techniques. This integration helps improve context, semantic understanding, and personalization, making Large Language Models (LLMs) more effective for critical applications.

We walk through an example that uses real-world customer and product data from a fashion, style, and beauty retailer. We show how you can use a knowledge graph to ground an LLM, enabling it to build tailored marketing content personalized to each customer based on their interests and shared purchase histories. We use Retrieval-Augmented Generation (RAG) to accomplish this,  specifically leveraging not just vector search but also graph pattern matching and graph machine learning to provide more relevant personalized results to customers. We call this graph-powered RAG approach “GraphRAG” for short.

This notebook walks through the first steps of the process, including:
- Building the knowledge graph and generating text embeddings from scratch 

*If you would rather start from a database dump and skip data loading, please skip to [genai-workshop.ipynb](https://github.com/neo4j-product-examples/genai-workshop/blob/main/genai-workshop.ipynb)* 



Please also see the following companion notebooks to complete the workshop: 

- [genai-workshop.ipynb](https://github.com/neo4j-product-examples/genai-workshop/blob/main/genai-workshop.ipynb)
    - Vector search 
    - Using graph patterns in Cypher to improve semantic search with context
    - Further augmenting semantic search with knowledge graph inference & graph data science    
    - Building the LLM chain and demo app for generating content 
    
- [genai-example-app-only](https://github.com/neo4j-product-examples/genai-workshop/blob/main/genai-workshop-app-only.ipynb)
    - Building the LLM chain and demo app for generating content 
    

### Some Logics
1. Run the pip install below to get the necessary dependencies.  this can take a while. Then run the following cell to import relevant libraries
2. You will need a Neo4j database environment with the [graph data science library](https://neo4j.com/docs/graph-data-science/current/installation) installed e.g. 
    - [AuraDS](https://neo4j.com/docs/aura/aurads/) 
    - [Neo4j Desktop](https://neo4j.com/docs/browser-manual/current/deployment-modes/neo4j-desktop/) 
    - Your own server environment 

In [3]:
%%capture
%pip install sentence_transformers langchain langchain-openai langchain_community openai tiktoken python-dotenv gradio graphdatascience altair neo4j_tools
%pip install "vegafusion[embed]"

In [4]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from graphdatascience import GraphDataScience
from neo4j_tools import gds_db_load, gds_utils
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.graphs import Neo4jGraph
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnableLambda
import gradio as gr

In [5]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 0)

### Setup Credentials and Environment Variables

There are two things you need here.
1. Credentials to a Neo4j database with Graph Data Science (AuraDS, Neo4j Desktop, or your own environment)
2. Your own [OpenAI API key](https://platform.openai.com/docs/quickstart?context=python).  You can use [this one](https://docs.google.com/document/d/19Lqjd0MqRs088KUVnd23ZrVU9G0OAg-53U72VrFwwms/edit) if you do not have one already.

To make this easy, you can write the credentials and env variables directly into the below cell.

Alternatively, if you like, you can use an environments file. This is a best practice for the future, but fine to skip if you're just exploring.

In [7]:
# Neo4j
NEO4J_URI = 'copy_paste_your_db_uri_here' #change this
NEO4J_PASSWORD = 'terminologies-fire-planet' #change this
NEO4J_USERNAME = 'neo4j'
AURA_DS = True

# AI
LLM = 'gpt-4o'
os.environ['OPENAI_API_KEY'] = 'sk-...' #change this
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [9]:
# You can skip this cell if not using a ws.env file - alternative to above
from dotenv import load_dotenv
import os

if os.path.exists('ws.env'):
    load_dotenv('ws.env', override=True)

    # Neo4j
    NEO4J_URI = os.getenv('NEO4J_URI')
    NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
    NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
    AURA_DS = eval(os.getenv('AURA_DS').title())

    # AI
    LLM = 'gpt-4o'
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

## Knowledge Graph Building

<img src="img/hm-banner.png" alt="summary" width="2000"/>

We begin by building our knowledge graph. This workshop will leverage the [H&M Personalized Fashion Recommendations Dataset](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/data), a sample of real customer purchase data that includes rich information around products including names, types, descriptions, department sections, etc.

Below is the graph data model we will use:

<img src="img/data-model.png" alt="summary" width="1000"/>


### Connect to Neo4j

We will use the [Graph Data Science Python Client](https://neo4j.com/docs/graph-data-science-client/current/) to connect to Neo4j. This client makes it convenient to display results, as we will see later.  Perhaps more importantly, it allows us to easily run [Graph Data Science](https://neo4j.com/docs/graph-data-science/current/introduction/) algorithms from Python.

This client will only work if your Neo4j instance has Graph Data Science installed.  If not, you can still use the [Neo4j Python Driver](https://neo4j.com/docs/python-manual/current/) or use Langchain’s Neo4j Graph object that we will see later on.

In [10]:
# Use Neo4j URI and credentials according to our setup
gds = GraphDataScience(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD),
    aura_ds=AURA_DS)

# Necessary if you enabled Arrow on the db - this is true for AuraDS
gds.set_database("neo4j")

Test your connection by running the below.  It should output your GDS version.

In [12]:
gds.version()

'2.9.0+73'

### Get Source Data
This workshop will leverage the [H&M Personalized Fashion Recommendations Dataset](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/data), a sample of real customer purchase data that includes rich information around products including names, types, descriptions, department sections, etc.

*Bonus!*
The data we use is a slightly sampled and preformatted version of the kaggle data. If you are interested in what we did, you can find the details [here](https://github.com/neo4j-product-examples/genai-workshop/blob/main/data-prep.ipynb)

In [13]:
import pandas as pd

# get source data - it has been pre-formatted. If you would like to re-generate from source on Kaggle, see the data-prep.ipynb notebook
department_df = pd.read_csv('https://storage.googleapis.com/neo4j-workshop-data/genai-hm/department.csv')
product_df = pd.read_csv('https://storage.googleapis.com/neo4j-workshop-data/genai-hm/product.csv')
article_df = pd.read_csv('https://storage.googleapis.com/neo4j-workshop-data/genai-hm/article.csv')
customer_df = pd.read_csv('https://storage.googleapis.com/neo4j-workshop-data/genai-hm/customer.csv')
transaction_df = pd.read_csv('https://storage.googleapis.com/neo4j-workshop-data/genai-hm/transaction.csv')

### Create Constraints

Before loading data into Neo4j, it is usually best practice to create Key or Uniqueness constraints for nodes. These [constraints](https://neo4j.com/docs/cypher-manual/current/constraints/) act as an index with some validation on unique id properties and thus make `MATCH` statements run significantly faster. Not doing this can result in a VERY slow ingest, so this is a critical step.

In [14]:
# create constraints - one uniqueness constraint for each node label
gds.run_cypher('CREATE CONSTRAINT unique_department_no IF NOT EXISTS FOR (n:Department) REQUIRE n.departmentNo IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_product_code IF NOT EXISTS FOR (n:Product) REQUIRE n.productCode IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_article_id IF NOT EXISTS FOR (n:Article) REQUIRE n.articleId IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_customer_id IF NOT EXISTS FOR (n:Customer) REQUIRE n.customerId IS UNIQUE')

""


### Loading Data with Helper Functions

Since we normalized our data beforehand, we can load each node and relationship type separately in batches.
The Node and Relationship query patterns will follow the same template for different types. The below functions simply automatically construct the queries and handle the batching.  They will print the queries they are using while loading so you can see the patterns.

Cypher for Loading Nodes follows a MATCH-MERGE pattern, while Cypher for loading relationships follows a MATCH-MATCH-MERGE pattern.

In [16]:
from typing import Tuple, Union
from numpy.typing import ArrayLike


def make_map(x):
    if type(x) == str:
        return x, x
    elif type(x) == tuple:
        return x
    else:
        raise Exception("Entry must of type string or tuple")


def make_set_clause(prop_names: ArrayLike, element_name='n', item_name='rec'):
    clause_list = []
    for prop_name in prop_names:
        clause_list.append(f'{element_name}.{prop_name} = {item_name}.{prop_name}')
    return 'SET ' + ', '.join(clause_list)


def make_node_merge_query(node_key_name: str, node_label: str, cols: ArrayLike):
    template = f'''UNWIND $recs AS rec\nMERGE(n:{node_label} {{{node_key_name}: rec.{node_key_name}}})'''
    prop_names = [x for x in cols if x != node_key_name]
    if len(prop_names) > 0:
        template = template + '\n' + make_set_clause(prop_names)
    return template + '\nRETURN count(n) AS nodeLoadedCount'


def make_rel_merge_query(source_target_labels: Union[Tuple[str, str], str],
                         source_node_key: Union[Tuple[str, str], str],
                         target_node_key: Union[Tuple[str, str], str],
                         rel_type: str,
                         cols: ArrayLike,
                         rel_key: str = None):
    source_target_label_map = make_map(source_target_labels)
    source_node_key_map = make_map(source_node_key)
    target_node_key_map = make_map(target_node_key)

    merge_statement = f'MERGE(s)-[r:{rel_type}]->(t)'
    if rel_key is not None:
        merge_statement = f'MERGE(s)-[r:{rel_type} {{{rel_key}: rec.{rel_key}}}]->(t)'

    template = f'''\tUNWIND $recs AS rec
    MATCH(s:{source_target_label_map[0]} {{{source_node_key_map[0]}: rec.{source_node_key_map[1]}}})
    MATCH(t:{source_target_label_map[1]} {{{target_node_key_map[0]}: rec.{target_node_key_map[1]}}})\n\t''' + merge_statement
    prop_names = [x for x in cols if x not in [rel_key, source_node_key_map[1], target_node_key_map[1]]]
    if len(prop_names) > 0:
        template = template + '\n\t' + make_set_clause(prop_names, 'r')
    return template + '\n\tRETURN count(r) AS relLoadedCount'


def chunks(xs, n=10_000):
    n = max(1, n)
    return [xs[i:i + n] for i in range(0, len(xs), n)]


def load_nodes(gds: GraphDataScience, node_df: pd.DataFrame, node_key_col: str, node_label: str, chunk_size=10_000):
    records = node_df.to_dict('records')
    print(f'======  loading {node_label} nodes  ======')
    total = len(records)
    print(f'staging {total:,} records')
    query = make_node_merge_query(node_key_col, node_label, node_df.columns.copy())
    print(f'\nUsing This Cypher Query:\n```\n{query}\n```\n')
    cumulative_count = 0
    for recs in chunks(records, chunk_size):
        res = gds.run_cypher(query, params={'recs': recs})
        cumulative_count += res.iloc[0, 0]
        print(f'Loaded {cumulative_count:,} of {total:,} nodes')


def load_rels(gds: GraphDataScience,
              rel_df: pd.DataFrame,
              source_target_labels: Union[Tuple[str, str], str],
              source_node_key: Union[Tuple[str, str], str],
              target_node_key: Union[Tuple[str, str], str],
              rel_type: str,
              rel_key: str = None,
              chunk_size=10_000):
    records = rel_df.to_dict('records')
    print(f'======  loading {rel_type} relationships  ======')
    total = len(records)
    print(f'staging {total:,} records')
    query = make_rel_merge_query(source_target_labels, source_node_key,
                                 target_node_key, rel_type, rel_df.columns.copy(), rel_key)
    print(f'\nUsing This Cypher Query:\n```\n{query}\n```\n')
    cumulative_count = 0
    for recs in chunks(records, chunk_size):
        res = gds.run_cypher(query, params={'recs': recs})
        cumulative_count += res.iloc[0, 0]
        print(f'Loaded {cumulative_count:,} of {total:,} relationships')

In [17]:
%%time

# load nodes
load_nodes(gds, department_df, 'departmentNo', 'Department')
load_nodes(gds, article_df.drop(columns=['productCode', 'departmentNo']), 'articleId', 'Article')
load_nodes(gds, product_df, 'productCode', 'Product')
load_nodes(gds, customer_df, 'customerId', 'Customer')

# load relationships
load_rels(gds, article_df[['articleId', 'departmentNo']], source_target_labels=('Article', 'Department'),
                      source_node_key='articleId', target_node_key='departmentNo',
                      rel_type='FROM_DEPARTMENT')
load_rels(gds, article_df[['articleId', 'productCode']], source_target_labels=('Article', 'Product'),
                      source_node_key='articleId',target_node_key='productCode',
                      rel_type='VARIANT_OF')
load_rels(gds, transaction_df, source_target_labels=('Customer', 'Article'),
                      source_node_key='customerId', target_node_key='articleId', rel_key='txId',
                      rel_type='PURCHASED')

# convert transaction dates
gds.run_cypher('''
MATCH (:Customer)-[r:PURCHASED]->()
SET r.tDat = date(r.tDat)
''')

# convert NaN product descriptions
gds.run_cypher('''
MATCH (n:Product) WHERE valueType(n.detailDesc) <> "STRING NOT NULL"
SET n.detailDesc = ""
RETURN n
''')

# create combined text property. This will help simplify later with semantic search and RAG
gds.run_cypher("""
    MATCH(p:Product)
    SET p.text = 'Product-- ' +
        'Name: ' + p.prodName + ' || ' +
        'Type: ' + p.productTypeName + ' || ' +
        'Group: ' + p.productGroupName + ' || ' +
        'Garment Type: ' + p.garmentGroupName + ' || ' +
        'Description: ' + p.detailDesc
    RETURN count(p) AS propertySetCount
    """)

# write dummy urls to illustrate sourcing in future retrieval
gds.run_cypher("""
MATCH(p:Product)
SET p.url = 'https://representative-domain/product/' + p.productCode
""")

======  loading Department nodes  ======
staging 266 records

Using This Cypher Query:
```
UNWIND $recs AS rec
MERGE(n:Department {departmentNo: rec.departmentNo})
SET n.departmentName = rec.departmentName, n.sectionNo = rec.sectionNo, n.sectionName = rec.sectionName
RETURN count(n) AS nodeLoadedCount
```

Loaded 266 of 266 nodes
======  loading Article nodes  ======
staging 13,351 records

Using This Cypher Query:
```
UNWIND $recs AS rec
MERGE(n:Article {articleId: rec.articleId})
SET n.prodName = rec.prodName, n.productTypeName = rec.productTypeName, n.graphicalAppearanceNo = rec.graphicalAppearanceNo, n.graphicalAppearanceName = rec.graphicalAppearanceName, n.colourGroupCode = rec.colourGroupCode, n.colourGroupName = rec.colourGroupName
RETURN count(n) AS nodeLoadedCount
```

Loaded 10,000 of 13,351 nodes
Loaded 13,351 of 13,351 nodes
======  loading Product nodes  ======
staging 8,044 records

Using This Cypher Query:
```
UNWIND $recs AS rec
MERGE(n:Product {productCode: rec.produc

""


## Creating Text Embeddings & Vector Index

Now the data has been loaded, we need to generate text embeddings on our product nodes to support Vector Search

Neo4j has native integrations with popular embedding APIs (OpenAI, Vertex AI, Amazon Bedrock, Azure OpenAI) making it possible to generate embeddings with a single Cypher query using `genai.vector.*` operations*.

The below query embeds the Product text property with OpenAI `text-embedding-ada-002` in batches.  Specifically it
1. Matches every Product that has a detailed description
2. Uses the `collect` aggregation function to batch products into a set number of partitions
3. Encodes the text property in batches using OpenAI `text-embedding-ada-002`
4. Sets the embedding as a vector property using `db.create.setNodeVectorProperty`. This special function is used to set the properties as floats rather than double precision, which requires more space.  This becomes important as these embedding vectors tend to be long, and the size can add up quickly.

*NOTE: `genai.vector.*` operations are not available in Neo4j Community Edition.  For Neo4j Community Edition you will need to generate embeddings externally and ingest them into Neo4j.

In [18]:
#generate embeddings

gds.run_cypher('''
MATCH (n:Product) WHERE size(n.detailDesc) <> 0
WITH collect(n) AS nodes, toInteger(rand()*$numberOfBatches) AS partition
CALL {
    WITH nodes
    CALL genai.vector.encodeBatch([node IN nodes| node.text], "OpenAI", { token: $token})
    YIELD index, vector
    CALL db.create.setNodeVectorProperty(nodes[index], "textEmbedding", vector)
} IN TRANSACTIONS OF 1 ROW''', params={'token':OPENAI_API_KEY, 'numberOfBatches':100})

""


After generating embeddings we will create a vector index for them. The Neo4j Vector Index enables efficient Approximate Nearest Neighbor (ANN) search with vectors. It uses the Hierarchical Navigable Small World (HNSW) algorithm.

The below cell will create the index, then, with a separate query, await for the index to come online, meaning it is ready to be used in vector search.

In [19]:
#create vector index

embedding_dimension = 1536 #default for OpenAI text-embedding-ada-002

gds.run_cypher('''
CREATE VECTOR INDEX product_text_embeddings IF NOT EXISTS FOR (n:Product) ON (n.textEmbedding)
OPTIONS {indexConfig: {
 `vector.dimensions`: toInteger($dimension),
 `vector.similarity_function`: 'cosine'
}}''', params={'dimension': embedding_dimension})

#wait for index to come online
gds.run_cypher('CALL db.awaitIndex("product_text_embeddings", 300)')

""


# Next Steps
Try out a vector search with your newly made vector index, and learn how to enhance that search with graphs and graph data science! [genai-workshop.ipynb](https://github.com/neo4j-product-examples/genai-workshop/blob/main/genai-workshop.ipynb)
